In [23]:
import pandas
import numpy as np
file_name = r'data/hackerrank_email/training_dataset.csv'
#names = [✬preg✬, ✬plas✬, ✬pres✬, ✬skin✬, ✬test✬, ✬mass✬, ✬pedi✬, ✬age✬, ✬class✬]
data = pandas.read_csv(file_name)
print(data.shape)

(486048, 54)


In [24]:
data.head(10)

,user_id,mail_id,mail_category,mail_type,sent_time,open_time,click_time,unsubscribe_time,last_online,hacker_created_at,...,submissions_count_contest_1_days,submissions_count_contest_30_days,submissions_count_contest_365_days,submissions_count_contest_7_days,submissions_count_master,submissions_count_master_1_days,submissions_count_master_30_days,submissions_count_master_365_days,submissions_count_master_7_days,unsubscribed
0,AR+tMy3H/E+Re8Id20zUIz+amJkv6KU12o+BrgIDin0=,DQ/4I+GIOz2ZoIiK0Lg0AkwnI35XotghgUK/MYc101I=,mail_category_2,mail_type_1,1463497837,1.463541e+09,NaN,NaN,1.459520e+09,1432533023,...,0,0,0,0,13,0,0,13,0,False
1,1P4AOvdzJzhDSHi7jJ3udWv4ajpKxOn4T/rCLv4PrXU=,BL3z4RtiyfIDydaRYWX2ZXL6IX10QH1yG5ak1s/8Lls=,mail_category_1,mail_type_1,1461357640,NaN,NaN,NaN,1.461210e+09,1432184291,...,0,3,16,0,83,0,43,83,4,False
2,OEfFUcsTAGInCfsHuLZuIgdSNtuNsg8EdfN98VUZVTs=,BL3z4RtiyfIDydaRYWX2ZXL6IX10QH1yG5ak1s/8Lls=,mail_category_1,mail_type_1,1463499639,NaN,NaN,NaN,1.463411e+09,1433045937,...,0,0,0,0,16,0,3,16,0,False
3,1P4AOvdzJzhDSHi7jJ3udWv4ajpKxOn4T/rCLv4PrXU=,EHNBRbi6i9KO6cMHsuDPFjZVp2cY3RH+BiOKwPwzLQs=,mail_category_1,mail_type_1,1463182983,NaN,NaN,NaN,1.462768e+09,1432184291,...,0,0,16,0,85,0,9,85,0,False
4,CYRcuV0cR0algMZJ1N6+3uKcqi8iu+6tJNzmBbmgN7o=,K0y/NW59TJkYc5y0HUwDeAXrewYT0JQlkcozz0s2V5Q=,mail_category_4,mail_type_1,1461855019,NaN,NaN,NaN,1.461248e+09,1432998058,...,0,1,17,0,43,0,0,43,0,False
5,UREtRIcAzBFKmMt9ZSkFLofOWKAKmzsDD7BwrdxcPuk=,UGATDXARg7jMEInKH7oXgty2nwxnwD2l0OW/8Nsa0MI=,mail_category_6,mail_type_1,1459927021,NaN,NaN,NaN,1.457962e+09,1431945704,...,0,0,13,0,88,0,0,88,0,False
6,NiDHS4AU/QOs3ppF618O6bUg8BxJd+EV4Ek26KWrDc0=,qE9zgWiITYA97RUiN4X/t9IVWLViLz+lKijaYegyBiQ=,mail_category_1,mail_type_1,1460230930,1.460232e+09,NaN,NaN,1.460135e+09,1432658834,...,0,0,0,0,20,0,12,20,12,False
7,sGNmP4gGVwh3YBpXtq6ZI05PIa3Z0iUaF+ujKonctu0=,BL3z4RtiyfIDydaRYWX2ZXL6IX10QH1yG5ak1s/8Lls=,mail_category_1,mail_type_1,1462282973,NaN,NaN,NaN,1.462193e+09,1432294499,...,0,0,0,0,14,0,14,14,14,False
8,nyoFu8qU0B5viJDXfrVRQ65ytOSSSY3c7okcjo1Z76M=,4+EEK8RbNYwuFCHznY9XSRCV4Yek60bHVgnP3jtjjzk=,mail_category_13,mail_type_2,1462888545,1.462890e+09,NaN,NaN,1.462860e+09,1432795707,...,0,19,265,0,129,0,0,129,0,False
9,A2EUbD7QN5SXpJ7FhPY5Ne7zwjwYhRvAheGskLl0Os8=,4+EEK8RbNYwuFCHznY9XSRCV4Yek60bHVgnP3jtjjzk=,mail_category_13,mail_type_2,1462888549,NaN,NaN,NaN,1.462369e+09,1432059719,...,0,0,0,0,3,0,0,3,0,False


In [41]:
 data_subset = data[['mail_category', 'mail_type','sent_time','submissions_count_contest_1_days','submissions_count_contest_30_days',
                     'submissions_count_contest_365_days','submissions_count_contest_7_days','submissions_count_master',
                     'submissions_count_master_1_days','submissions_count_master_30_days',
                     'submissions_count_master_365_days','submissions_count_master_7_days','unsubscribed',
'contest_login_count_1_days','contest_login_count_30_days','contest_login_count_365_days','contest_login_count_7_days',
'contest_participation_count','contest_participation_count_1_days','contest_participation_count_30_days',
'contest_participation_count_365_days','contest_participation_count_7_days','forum_comments_count',
'forum_count','forum_expert_count','forum_questions_count',
'hacker_confirmation','ipn_count','ipn_count_1_days','ipn_count_30_days','ipn_count_365_days',
'ipn_count_7_days','ipn_read','ipn_read_1_days','ipn_read_30_days',
'ipn_read_365_days','ipn_read_7_days','submissions_count','opened']]

# # data_subset = data[['mail_category', 'mail_type','sent_time','submissions_count_contest_1_days','submissions_count_contest_30_days','submissions_count_contest_365_days','submissions_count_contest_7_days','submissions_count_master','submissions_count_master_1_days','submissions_count_master_30_days','submissions_count_master_365_days','submissions_count_master_7_days','unsubscribed'
# #                     ,'opened']]

# col_names = data.columns.values
# # for name in data.columns.values:
# #     col_names.append(name)

# #print(col_names)
# data_subset = data[data[[data.column for data.column in col_names]]]
data_subset.shape

(486048, 39)

In [45]:
 data_subset1 = data[['mail_category', 'mail_type','sent_time','submissions_count_contest_1_days','submissions_count_contest_30_days',
                     'submissions_count_contest_365_days','submissions_count_contest_7_days','submissions_count_master',
                     'submissions_count_master_1_days','submissions_count_master_30_days',
                     'submissions_count_master_365_days','submissions_count_master_7_days','unsubscribed'
# 'contest_login_count_1_days','contest_login_count_30_days','contest_login_count_365_days','contest_login_count_7_days',
# 'contest_participation_count','contest_participation_count_1_days','contest_participation_count_30_days',
# 'contest_participation_count_365_days','contest_participation_count_7_days','forum_comments_count',
# 'forum_count','forum_expert_count','forum_questions_count',
# 'hacker_confirmation','ipn_count','ipn_count_1_days','ipn_count_30_days','ipn_count_365_days',
# 'ipn_count_7_days','ipn_read','ipn_read_1_days','ipn_read_30_days',
# 'ipn_read_365_days','ipn_read_7_days','submissions_count'
                      ,'opened']]

# # data_subset = data[['mail_category', 'mail_type','sent_time','submissions_count_contest_1_days','submissions_count_contest_30_days','submissions_count_contest_365_days','submissions_count_contest_7_days','submissions_count_master','submissions_count_master_1_days','submissions_count_master_30_days','submissions_count_master_365_days','submissions_count_master_7_days','unsubscribed'
# #                     ,'opened']]

# col_names = data.columns.values
# # for name in data.columns.values:
# #     col_names.append(name)

# #print(col_names)
# data_subset = data[data[[data.column for data.column in col_names]]]
data_subset1.corr(method='pearson')

,sent_time,submissions_count_contest_1_days,submissions_count_contest_30_days,submissions_count_contest_365_days,submissions_count_contest_7_days,submissions_count_master,submissions_count_master_1_days,submissions_count_master_30_days,submissions_count_master_365_days,submissions_count_master_7_days,unsubscribed,opened
sent_time,1.000000,0.031244,-0.017486,-0.062436,0.010901,-0.056077,0.000898,-0.007932,-0.057513,-0.026443,-0.001525,-0.077610
submissions_count_contest_1_days,0.031244,1.000000,0.267686,0.142411,0.408485,0.067450,0.052486,0.067947,0.067792,0.058422,-0.002566,0.018085
submissions_count_contest_30_days,-0.017486,0.267686,1.000000,0.499043,0.658388,0.224128,0.051274,0.191884,0.229400,0.091534,-0.007157,0.023940
submissions_count_contest_365_days,-0.062436,0.142411,0.499043,1.000000,0.322640,0.431898,0.014591,0.034491,0.402062,0.004392,-0.011043,0.003038
submissions_count_contest_7_days,0.010901,0.408485,0.658388,0.322640,1.000000,0.140742,0.071824,0.132130,0.143329,0.104181,-0.006327,0.020139
submissions_count_master,-0.056077,0.067450,0.224128,0.431898,0.140742,1.000000,0.088334,0.387960,0.931771,0.194368,-0.016530,0.004923
submissions_count_master_1_days,0.000898,0.052486,0.051274,0.014591,0.071824,0.088334,1.000000,0.226566,0.102532,0.377407,-0.003728,0.028692
submissions_count_master_30_days,-0.007932,0.067947,0.191884,0.034491,0.132130,0.387960,0.226566,1.000000,0.454039,0.589156,-0.007280,0.051546
submissions_count_master_365_days,-0.057513,0.067792,0.229400,0.402062,0.143329,0.931771,0.102532,0.454039,1.000000,0.231155,-0.017188,0.007116
submissions_count_master_7_days,-0.026443,0.058422,0.091534,0.004392,0.104181,0.194368,0.377407,0.589156,0.231155,1.000000,-0.003167,0.054511


In [46]:
 data_subset2 = data[[
#             'mail_category', 'mail_type','sent_time','submissions_count_contest_1_days','submissions_count_contest_30_days',
#                      'submissions_count_contest_365_days','submissions_count_contest_7_days','submissions_count_master',
#                      'submissions_count_master_1_days','submissions_count_master_30_days',
#                      'submissions_count_master_365_days','submissions_count_master_7_days','unsubscribed'
'contest_login_count_1_days','contest_login_count_30_days','contest_login_count_365_days','contest_login_count_7_days',
'contest_participation_count','contest_participation_count_1_days','contest_participation_count_30_days',
'contest_participation_count_365_days','contest_participation_count_7_days','forum_comments_count'
# 'forum_count','forum_expert_count','forum_questions_count',
# 'hacker_confirmation','ipn_count','ipn_count_1_days','ipn_count_30_days','ipn_count_365_days',
# 'ipn_count_7_days','ipn_read','ipn_read_1_days','ipn_read_30_days',
# 'ipn_read_365_days','ipn_read_7_days','submissions_count'
                      ,'opened']]

# # data_subset = data[['mail_category', 'mail_type','sent_time','submissions_count_contest_1_days','submissions_count_contest_30_days','submissions_count_contest_365_days','submissions_count_contest_7_days','submissions_count_master','submissions_count_master_1_days','submissions_count_master_30_days','submissions_count_master_365_days','submissions_count_master_7_days','unsubscribed'
# #                     ,'opened']]

# col_names = data.columns.values
# # for name in data.columns.values:
# #     col_names.append(name)

# #print(col_names)
# data_subset = data[data[[data.column for data.column in col_names]]]
data_subset2.corr(method='pearson')

,contest_login_count_1_days,contest_login_count_30_days,contest_login_count_365_days,contest_login_count_7_days,contest_participation_count,contest_participation_count_1_days,contest_participation_count_30_days,contest_participation_count_365_days,contest_participation_count_7_days,forum_comments_count,opened
contest_login_count_1_days,1.000000,0.185927,0.107816,0.279734,0.089315,0.776164,0.151320,0.098981,0.224788,0.023490,0.012094
contest_login_count_30_days,0.185927,1.000000,0.405735,0.546756,0.312743,0.169096,0.784348,0.361385,0.451086,0.098153,0.048865
contest_login_count_365_days,0.107816,0.405735,1.000000,0.183727,0.882951,0.106349,0.403735,0.951066,0.197353,0.280944,-0.004225
contest_login_count_7_days,0.279734,0.546756,0.183727,1.000000,0.131405,0.234260,0.395219,0.153574,0.761096,0.032892,0.043133
contest_participation_count,0.089315,0.312743,0.882951,0.131405,1.000000,0.099588,0.395352,0.927068,0.189886,0.266551,-0.010031
contest_participation_count_1_days,0.776164,0.169096,0.106349,0.234260,0.099588,1.000000,0.197216,0.111488,0.297892,0.022912,0.013192
contest_participation_count_30_days,0.151320,0.784348,0.403735,0.395219,0.395352,0.197216,1.000000,0.455376,0.555723,0.109041,0.032987
contest_participation_count_365_days,0.098981,0.361385,0.951066,0.153574,0.927068,0.111488,0.455376,1.000000,0.220409,0.290203,-0.009380
contest_participation_count_7_days,0.224788,0.451086,0.197353,0.761096,0.189886,0.297892,0.555723,0.220409,1.000000,0.041959,0.035827
forum_comments_count,0.023490,0.098153,0.280944,0.032892,0.266551,0.022912,0.109041,0.290203,0.041959,1.000000,-0.005169


In [47]:
 data_subset3 = data[[
#             'mail_category', 'mail_type','sent_time','submissions_count_contest_1_days','submissions_count_contest_30_days',
#                      'submissions_count_contest_365_days','submissions_count_contest_7_days','submissions_count_master',
#                      'submissions_count_master_1_days','submissions_count_master_30_days',
#                      'submissions_count_master_365_days','submissions_count_master_7_days','unsubscribed'
# 'contest_login_count_1_days','contest_login_count_30_days','contest_login_count_365_days','contest_login_count_7_days',
# 'contest_participation_count','contest_participation_count_1_days','contest_participation_count_30_days',
# 'contest_participation_count_365_days','contest_participation_count_7_days','forum_comments_count'
'forum_count','forum_expert_count','forum_questions_count',
'hacker_confirmation','ipn_count','ipn_count_1_days','ipn_count_30_days','ipn_count_365_days',
'ipn_count_7_days','ipn_read','ipn_read_1_days','ipn_read_30_days',
'ipn_read_365_days','ipn_read_7_days','submissions_count'
                      ,'opened']]

# # data_subset = data[['mail_category', 'mail_type','sent_time','submissions_count_contest_1_days','submissions_count_contest_30_days','submissions_count_contest_365_days','submissions_count_contest_7_days','submissions_count_master','submissions_count_master_1_days','submissions_count_master_30_days','submissions_count_master_365_days','submissions_count_master_7_days','unsubscribed'
# #                     ,'opened']]

# col_names = data.columns.values
# # for name in data.columns.values:
# #     col_names.append(name)

# #print(col_names)
# data_subset = data[data[[data.column for data.column in col_names]]]
data_subset3.corr(method='pearson')

,forum_count,forum_expert_count,forum_questions_count,hacker_confirmation,ipn_count,ipn_count_1_days,ipn_count_30_days,ipn_count_365_days,ipn_count_7_days,ipn_read,ipn_read_1_days,ipn_read_30_days,ipn_read_365_days,ipn_read_7_days,submissions_count,opened
forum_count,1.000000,0.340293,0.541159,0.041614,0.420523,0.055234,0.100483,0.415643,0.067035,0.359101,0.041081,0.088955,0.333705,0.056663,0.436746,-0.007138
forum_expert_count,0.340293,1.000000,0.200940,0.026461,0.183686,0.012584,0.035457,0.185030,0.018001,0.164165,0.016308,0.050301,0.163054,0.026885,0.167015,0.004050
forum_questions_count,0.541159,0.200940,1.000000,0.036686,0.333055,0.042931,0.082869,0.334649,0.053834,0.273307,0.032837,0.070407,0.273190,0.046124,0.290342,-0.008493
hacker_confirmation,0.041614,0.026461,0.036686,1.000000,0.168122,0.037712,0.063809,0.169756,0.041209,0.094110,0.020016,0.052651,0.092125,0.034797,0.105566,0.076434
ipn_count,0.420523,0.183686,0.333055,0.168122,1.000000,0.134571,0.372289,0.996381,0.202219,0.600081,0.029940,0.123558,0.560000,0.050414,0.635810,-0.016435
ipn_count_1_days,0.055234,0.012584,0.042931,0.037712,0.134571,1.000000,0.408601,0.137938,0.674870,0.069787,0.270808,0.191858,0.073511,0.223477,0.056966,-0.023656
ipn_count_30_days,0.100483,0.035457,0.082869,0.063809,0.372289,0.408601,1.000000,0.382147,0.691542,0.163110,0.080029,0.356279,0.171613,0.177299,0.133369,-0.035852
ipn_count_365_days,0.415643,0.185030,0.334649,0.169756,0.996381,0.137938,0.382147,1.000000,0.208024,0.588961,0.030811,0.127535,0.563843,0.052396,0.620399,-0.016643
ipn_count_7_days,0.067035,0.018001,0.053834,0.041209,0.202219,0.674870,0.691542,0.208024,1.000000,0.097951,0.148846,0.302181,0.104635,0.308971,0.073451,-0.034826
ipn_read,0.359101,0.164165,0.273307,0.094110,0.600081,0.069787,0.163110,0.588961,0.097951,1.000000,0.126509,0.390977,0.976844,0.235124,0.526064,0.021566


In [42]:
len(data_subset[data_subset['opened'] == True])

161347

In [43]:
len(data_subset[data_subset['opened'] == False])

324701

In [44]:
data_subset.corr(method='pearson')

,sent_time,submissions_count_contest_1_days,submissions_count_contest_30_days,submissions_count_contest_365_days,submissions_count_contest_7_days,submissions_count_master,submissions_count_master_1_days,submissions_count_master_30_days,submissions_count_master_365_days,submissions_count_master_7_days,...,ipn_count_30_days,ipn_count_365_days,ipn_count_7_days,ipn_read,ipn_read_1_days,ipn_read_30_days,ipn_read_365_days,ipn_read_7_days,submissions_count,opened
sent_time,1.000000,0.031244,-0.017486,-0.062436,0.010901,-0.056077,0.000898,-0.007932,-0.057513,-0.026443,...,0.110410,-0.011742,0.097866,-0.050141,-0.006665,-0.026670,-0.047693,-0.015396,-0.060353,-0.077610
submissions_count_contest_1_days,0.031244,1.000000,0.267686,0.142411,0.408485,0.067450,0.052486,0.067947,0.067792,0.058422,...,0.026819,0.056077,0.046021,0.059953,0.106671,0.046876,0.058377,0.078817,0.106967,0.018085
submissions_count_contest_30_days,-0.017486,0.267686,1.000000,0.499043,0.658388,0.224128,0.051274,0.191884,0.229400,0.091534,...,0.102150,0.192068,0.080447,0.195265,0.059664,0.146599,0.187603,0.109616,0.365361,0.023940
submissions_count_contest_365_days,-0.062436,0.142411,0.499043,1.000000,0.322640,0.431898,0.014591,0.034491,0.402062,0.004392,...,0.071380,0.563074,0.039816,0.495369,0.030834,0.071683,0.475611,0.048385,0.726056,0.003038
submissions_count_contest_7_days,0.010901,0.408485,0.658388,0.322640,1.000000,0.140742,0.071824,0.132130,0.143329,0.104181,...,0.062637,0.122653,0.099915,0.126323,0.077078,0.100770,0.122116,0.146561,0.233598,0.020139
submissions_count_master,-0.056077,0.067450,0.224128,0.431898,0.140742,1.000000,0.088334,0.387960,0.931771,0.194368,...,0.183787,0.575923,0.103333,0.428702,0.055707,0.163646,0.414164,0.092851,0.803715,0.004923
submissions_count_master_1_days,0.000898,0.052486,0.051274,0.014591,0.071824,0.088334,1.000000,0.226566,0.102532,0.377407,...,0.039186,0.007395,0.063181,0.024524,0.107201,0.078600,0.027240,0.111051,0.053966,0.028692
submissions_count_master_30_days,-0.007932,0.067947,0.191884,0.034491,0.132130,0.387960,0.226566,1.000000,0.454039,0.589156,...,0.270913,0.046731,0.236448,0.095244,0.118179,0.342991,0.107427,0.229953,0.223542,0.051546
submissions_count_master_365_days,-0.057513,0.067792,0.229400,0.402062,0.143329,0.931771,0.102532,0.454039,1.000000,0.231155,...,0.196784,0.504298,0.110775,0.398195,0.063629,0.190491,0.413449,0.108859,0.692548,0.007116
submissions_count_master_7_days,-0.026443,0.058422,0.091534,0.004392,0.104181,0.194368,0.377407,0.589156,0.231155,1.000000,...,0.087763,-0.019175,0.172285,0.031359,0.138916,0.174877,0.038274,0.249636,0.104709,0.054511


In [4]:
list(data.columns.values)

['user_id',
 'mail_id',
 'mail_category',
 'mail_type',
 'sent_time',
 'open_time',
 'click_time',
 'unsubscribe_time',
 'last_online',
 'hacker_created_at',
 'hacker_timezone',
 'clicked',
 'contest_login_count',
 'contest_login_count_1_days',
 'contest_login_count_30_days',
 'contest_login_count_365_days',
 'contest_login_count_7_days',
 'contest_participation_count',
 'contest_participation_count_1_days',
 'contest_participation_count_30_days',
 'contest_participation_count_365_days',
 'contest_participation_count_7_days',
 'forum_comments_count',
 'forum_count',
 'forum_expert_count',
 'forum_questions_count',
 'hacker_confirmation',
 'ipn_count',
 'ipn_count_1_days',
 'ipn_count_30_days',
 'ipn_count_365_days',
 'ipn_count_7_days',
 'ipn_read',
 'ipn_read_1_days',
 'ipn_read_30_days',
 'ipn_read_365_days',
 'ipn_read_7_days',
 'opened',
 'submissions_count',
 'submissions_count_1_days',
 'submissions_count_30_days',
 'submissions_count_365_days',
 'submissions_count_7_days',
 'sub

In [5]:
data.dtypes

user_id                                  object
mail_id                                  object
mail_category                            object
mail_type                                object
sent_time                                 int64
open_time                               float64
click_time                              float64
unsubscribe_time                        float64
last_online                             float64
hacker_created_at                         int64
hacker_timezone                         float64
clicked                                    bool
contest_login_count                       int64
contest_login_count_1_days                int64
contest_login_count_30_days               int64
contest_login_count_365_days              int64
contest_login_count_7_days                int64
contest_participation_count               int64
contest_participation_count_1_days        int64
contest_participation_count_30_days       int64
contest_participation_count_365_days    